Copyright (c) Microsoft Corporation. All rights reserved.
Licensed under the MIT License.

# Train your own Model and Deploy to Device

**NOTE**
* Warning: copying *.pb, *.bin, or, *.blob using the web interface can corrupt the files. If needed download and use Azure storage explorer or the CL.
* Make sure to run each cell individually as some cells will require input parameters.

This notebook shows how to create a Tensorflow object detection model, how to convert the model to the appropriate format for the Eye development
kit, and how to deploy the model to your kit.

This notebook takes a transfer learning approach, using a pre-trained Tensorflow Mobilenet model with custom dataset
and SSDLite layers that we will train to detect bowls.
We use the [Tensorflow object detection API](https://github.com/tensorflow/models/tree/master/research/object_detection).

The trained model will be deployed to Azure Eye Devkit using Module Twin Update method.

In [ ]:
# Save current directory for later reference
modelroot = !pwd
modelroot = modelroot[0]
modelroot

In [ ]:
# Setup workspace for Azure ML
!pip3 install azureml.core

import azureml.core
from azureml.core import Workspace
print(azureml.core.VERSION)

## OpenVINO

At this point, you should have a model exported from Tensorflow and it should be providing good bounding boxes on bowls.
The rest of this notebook will show you how to convert the model into the format that the EyeSOM dev kit requires,
and then how to download it to your device.

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
interactive_auth = InteractiveLoginAuthentication(tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47")

# If you have an Azure config.json in your workspace, you can use this line
ws = Workspace.from_config(auth=interactive_auth)

# Otherwise, you can manually fill in your information and use the next lines
#ws = Workspace(subscription_id="<YOUR SUBSCRIPTION ID>",
#                resource_group="<YOUR RESOURCE GROUP>",
#                workspace_name="<YOUR WORKSPACE NAME>",
#                auth=interactive_auth)

ws.get_details()
print(ws)

In [ ]:
# Reload workspace to register model
from azureml.core import Workspace
ws.get_details()

In [ ]:
# Register the trained model. Once registered, you'll find the model in the Models section on the left pane
from azureml.core.model import Model

trained_model_path = modelroot + '/data/fine_tuned_model/'
print("Trained model path:", trained_model_path)

model = Model.register(model_path=trained_model_path,
                      model_name="vehicle_ssdv2lite",
                      tags={"data": "ssd_mobilenetv2lite", "model": "object_detection", "type": "ssd_mobilenetv2lite"},
                      description="Retrained vehicle detection based on ssd_mobilenetv2lite",
                      workspace=ws)
model_name = str(model.name)
model_version = str(model.version)
print('Name: ' + model_name)
print('Version: ' + model_version)

In [ ]:
# Encrypt the model with Secure AI lifecycle SDK 
%cd $modelroot/data

import sczpy

os.environ["AZURE_CLIENT_ID"] = "adf49b19-baca-4aeb-b7ee-092856693f32"
os.environ["AZURE_CLIENT_SECRET"] = "c3832e94-787f-4a57-b30e-1a2d999ebd1b"
os.environ["AZURE_TENANT_ID"] = "72f988bf-86f1-41af-91ab-2d7cd011db47"
server_url = "https://pe-mm1.westus2.cloudapp.azure.com"

client = sczpy.SCZClient(server_url)
client.register_model(model_name, model_version)
client.encrypt(model_name, model_version, "model.zip", "model.enc.zip")
os.remove("model.zip")
os.rename("model.enc.zip", "model.zip")
client.upload_model(model_name, model_version, "model.zip")

In [ ]:
#Reload workspace details for module twin update
from azureml.core import Workspace
ws.get_details()

In [ ]:
# Get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

In [ ]:
# Set data path for model.zip and upload
data_path = 'model'
ds.upload(src_dir='.', target_path=data_path, overwrite=True)

In [ ]:
!pip3 install azure-storage-blob==2.1.0
!pip3 install msrest

In [ ]:
# Generate download SAS URL for model.zip
from datetime import datetime, timedelta
from azure.storage.blob import (
    BlockBlobService,
    ContainerPermissions,
    BlobPermissions,
    PublicAccess,
)

AZURE_ACC_NAME = ds.account_name
AZURE_PRIMARY_KEY = ds.account_key
AZURE_CONTAINER = ds.container_name
AZURE_BLOB=ds.name
AZURE_File=data_path+'/model.zip'

block_blob_service = BlockBlobService(account_name=AZURE_ACC_NAME, account_key=AZURE_PRIMARY_KEY)
sas_url = block_blob_service.generate_blob_shared_access_signature(AZURE_CONTAINER,AZURE_File,permission=BlobPermissions.READ,expiry= datetime.utcnow() + timedelta(hours=360))
downloadurl ='https://'+AZURE_ACC_NAME+'.blob.core.windows.net/'+AZURE_CONTAINER+'/'+AZURE_File+'?'+sas_url
print('https://'+AZURE_ACC_NAME+'.blob.core.windows.net/'+AZURE_CONTAINER+'/'+AZURE_File+'?'+sas_url)
print(sas_url)

In [ ]:
# Perform Module twin update
# Incorporate the connection string, device_id and the module_id values from your IoTHub

!pip3 install azure-iot-hub
print(downloadurl)
import sys
from azure.iot.hub import IoTHubRegistryManager
from azure.iot.hub.models import Twin, TwinProperties

# Incorporate Iothub connection string and the default module name
# Go to Https://portal.azure.com
# Select your IoTHub
# Click on Shared access policies
# Click service on right
# Copy the iothub connection string primary key

CONNECTION_STRING = 'HostName=PEBJIotHub.azure-devices.net;SharedAccessKeyName=service;SharedAccessKey=Ifa8KxdX3Qb0Kp9m02la1wVLQ+Cy2vlYpGM46w5tIiQ='
DEVICE_ID = 'devkit-0310-2b-1130'
MODULE_ID = "azureeyemodule"

try:
    # RegistryManager
    iothub_registry_manager = IoTHubRegistryManager(CONNECTION_STRING)

    module_twin = iothub_registry_manager.get_module_twin(DEVICE_ID, MODULE_ID)
    print ( "" )
    print ( "Module twin properties before update    :" )
    print ( "{0}".format(module_twin.properties) )

    # Update twin
    twin_patch = Twin()
    # Download model from AML model registry
    twin_patch.properties = TwinProperties(desired={"ModelZipUrl": downloadurl, "SecureAILifecycleEnabled": True, "SCZ_MODEL_NAME": model_name, "SCZ_MODEL_VERSION": model_version, "SCZ_MM_SERVER_URL": server_url, "DownloadSecuredModelFromMMServer": False})
    # Download model from MM server
    #twin_patch.properties = TwinProperties(desired={"SecureAILifecycleEnabled": True, "SCZ_MODEL_NAME": model_name, "SCZ_MODEL_VERSION": model_version, "SCZ_MM_SERVER_URL": server_url, "DownloadSecuredModelFromMMServer": True})
    updated_module_twin = iothub_registry_manager.update_module_twin(
        DEVICE_ID, MODULE_ID, twin_patch, module_twin.etag
    )
    print ( "" )
    print ( "Module twin properties after update     :" )
    print ( "{0}".format(updated_module_twin.properties) )

except Exception as ex:
    print ( "Unexpected error {0}".format(ex) )
except KeyboardInterrupt:
    print ( "IoTHubRegistryManager sample stopped" )

In [ ]:
# The trained model will get pushed to the IoT Edge device via module twin update method
# Check model inferencing by connecting monitor to the devkit or by installing VLC media player : 
#Install VLC from https://www.videolan.org/vlc/ and install on “Windows” to check the camera function of “Azure Eye”.

#Check video stream:
#1.	Select Media -> Open Network Stream…
#2.	Input the network stream: “rtsp://[ip of PE-101]:8554/result” then click “Play” button.
#3. or use webstream http://<ipaddressofcamer):3000

In [ ]:
os.remove("model.zip")
%cd ..